# Manejo de archivos en Python

## Manejo de rutas

### Problemática

Un problema común en el manejo de rutas es la incompatibilidad entre los sistemas de archivos de los sistemas operativos, ya sea Windows, Mac, Linux o WSL. Por ejemplo, Windows utiliza el “backslash” en sus rutas de archivos, mientras que el resto usa el “foward slash”.

#### Solución al manejo de rutas
Esto hace que, cuando inicies un nuevo proyecto, tengas que hacerte varias preguntas, por ejemplo:

* ¿Habrá más personas involucradas?
* ¿Habrá más de un ordenador involucrado?
* ¿Cuál será la ubicación del proyecto, dentro del sistema de archivos de cada ordenador?
* ¿Cómo se vería afectado un proyecto si reestructuras su contenido, en una fase intermedia de desarrollo?

Deberías poder trabajar en tu proyecto y no tener que preocuparte por nada más que eso.

### OS

#### Objetivo
Crear la ruta `./data/raw/` independiente del sistema operativo. En este caso usaremos os, un módulo de Python que sirve para manejar rutas.

**IMPORTANTE:** Cerciórate de que estás trabajando en el entorno correcto.

#### Implementación
Dentro del notebook de jupyter:

```python
import os

CURRENT_DIR = os.getcwd()  # Ruta actual de trabajo
DATA_DIR = os.path.join(CURRENT_DIR, os.pardir, "data", "raw")  # Ruta objetivo (os.pardir: ruta padre)

os.path.exists(DATA_DIR)  # Revisa si el path existe
os.path.isdir(DATA_DIR)  # Revisa si es un directorio

os.listdir(DATA_DIR)  # Itera por los archivos dentro del directorio

os.mkdir(os.path.join(DATA_DIR, "os"))  # Crea la carpeta *"os"*
```


### Pathlib

#### Objetivo
Crear la ruta `./data/raw/` independiente del sistema operativo. Ahora usaremos pathlib, otro módulo de Python.

#### Implementación
Dentro del notebook de jupyter:

``` python
import pathlib

pathlib.Path()  # Genera un objeto Unix Path o 

CURRENT_DIR = pathlib.Path().resolve()  # Path local completo
DATA_DIR = CURRENT_DIR.parent.joinpath("data", "raw")  # Directorio objetivo

DATA_DIR.exists()  # Revisa si el directorio existe
DATA_DIR.is_dir()  # Revisa si es un directorio
```

Utiliza el método `parent` para obtener el directorio padre y de ahí concatenar el path de las carpetas “data” y “raw”.

Puedes crear una carpeta dentro de un directorio, usando el método `mkdir`:

``` python
DATA_DIR.joinpath("<nombre_carpeta>").mkdir()
```

Para buscar la ruta de un archivo dentro del proyecto, usando `regex`:

``` python
list(DATA_DIR.glob("<nombre_archivo>"))
```

### PyFilesystem2

#### Objetivo
Crear la ruta `./data/raw/` independiente del sistema operativo. Ahora usaremos PyFilesystem2.

#### Implementación
Dentro del notebook de jupyter:

``` python
import fs

fs.open_fs(".")  # Abre una conexión con el path actual (OSFS)

CURRENT_DIR = fs.open_fs(".")

CURRENT_DIR.exists(".")  # Revisa si el directorio existe
DATA_DIR.listdir(".")  # Muestra el contenido dentro de la ruta.
```

* PyFilesystem2 genera un objeto OSFS (Operating System Filesystem).
* El inconveniente con este módulo es que el objeto OSFS solo detecta los objetos que existen en la ruta actual, por lo que si intentas acceder a un archivo ubicado en el directorio padre “…” te saltará un IndexError.
* Si necesitas que el objeto OSFS también detecte el directorio padre, además de las carpetas “data” y “raw”, vuelve a generar el objeto de la siguiente forma:

``` python
fs.open_fs("../data/raw/")  # Ruta objetivo
```

## Crear referencias relativas de archivos

### Objetivo
Necesitamos encontrar una forma de evitar que nuestro proyecto se rompa cuando movamos de lugar un archivo dentro del proyecto, para esto usaremos Referencias Relativas.

### Implementación
Usando PyProjRoot:

```python
import pyprojroot

pyprojroot.here()  # Esto es un Posix Path (pathlib)
pyprojroot.here().joinpath("data", "raw") 
```

* El path en pyprojroot se construye desde la raíz, no desde el path del archivo que lo ejecuta.
* Puedes mover el archivo a cualquier parte de la carpeta del proyecto, pero los paths no se romperán.

Usando PyHere:

```python
import pyhere

pyhere.here()  # También regresa un Posix Path
```

* El directorio que regresa es el directorio padre del directorio actual.

### Comparación
Estas dos líneas de código regresan el mismo resultado:

```python
pyprojroot.here("data").joinpath("raw")
pyhere.here().resolve() / "data" / "raw"
```

* Estas dos librerías sirven para crear shortcuts. Para esto, se puede usar la siguiente función:

```python
def make_dir_function(dir_name):
    def dir_function(*args):
        return pyprojroot.here()joinpath(dir_name, *args)
    return dir_function

data_dir = make_dir_function("data")
data_dir("raw", "pathlib")  # Devuelve el path personalizado
```

* Puedes crear la cantidad de shortcuts que tu proyecto necesite.